<a href="https://colab.research.google.com/github/Vanshika1703/twitter-sentiment-analysis/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import all the libraries
import pandas as pd
import numpy as np
import warnings
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
twitterdata=pd.read_csv('/content/Twitter_Data.csv')#path of dataset

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
print(stopwords.words('english'))#we have to remove these words as they do not add influential meaning to the data that's why they are not necessary

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
twitterdata.shape#no of rows and columns in the dataset

(32004, 2)

In [8]:
print(twitterdata)

                                              clean_text  category
0      when modi promised “minimum government maximum...      -1.0
1      talk all the nonsense and continue all the dra...       0.0
2      what did just say vote for modi  welcome bjp t...       1.0
3      asking his supporters prefix chowkidar their n...       1.0
4      answer who among these the most powerful world...       1.0
...                                                  ...       ...
31999    yup looks more like older version raga den modi       1.0
32000  modi have punished lots corrupt but cant remember      -1.0
32001  religion peace agenda clear hai chahe padhe li...       1.0
32002  chowkidar narendra modi was doing such stellar...       1.0
32003  joshi should fielded joint opposition candidat...       NaN

[32004 rows x 2 columns]


In [9]:
twitterdata.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [10]:
#counting the no of missing values in the dataset
twitterdata.isnull().sum()

,0
clean_text,1
category,1


In [11]:
#handling missing values
twitterdata=twitterdata.dropna()

In [12]:
#checking if there is another missing value left or not
twitterdata.isnull().sum()

,0
clean_text,0
category,0


In [13]:
twitterdata['category'].value_counts()

,count
category,
1.0,13569
0.0,10941
-1.0,7492


Stemming:
it
 is a process of reducing a word to its root word

In [14]:
1
port_stem=PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)#removing all the no and special character like@,# etc
  stemmed_content=stemmed_content.lower()#converting all the letters into lowercase
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [16]:
twitterdata['clean_text']=twitterdata['clean_text'].apply(stemming)

In [17]:
twitterdata.head()

,clean_text,category
0,modi promis minimum govern maximum govern expe...,-1.0
1,talk nonsens continu drama vote modi,0.0
2,say vote modi welcom bjp told rahul main campa...,1.0
3,ask support prefix chowkidar name modi great s...,1.0
4,answer among power world leader today trump pu...,1.0


In [18]:
x=twitterdata['clean_text'].values
y=twitterdata['category'].values

In [19]:
print(x)

['modi promis minimum govern maximum govern expect begin difficult job reform state take year get justic state busi exit psu templ'
 'talk nonsens continu drama vote modi'
 'say vote modi welcom bjp told rahul main campaign modi think modi relax'
 ... 'modi punish lot corrupt cant rememb'
 'religion peac agenda clear hai chahe padh likh anpad mazduri kart vote khilaf deng havent come across singl peopl religion peac will vote modi'
 'chowkidar narendra modi stellar job look anoth seat indian']


In [20]:
print(y)

[-1.  0.  1. ... -1.  1.  1.]


Splitting the data into train and test data

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [22]:
print(x.shape,x_train.shape,x_test.shape)

(32002,) (25601,) (6401,)


In [23]:
print(y.shape,y_train.shape,y_test.shape)

(32002,) (25601,) (6401,)


In [24]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x_train)
x_train=vectorizer.transform(x_train)
x_test=vectorizer.transform(x_test)

In [25]:
print(x_train)

  (0, 2280)	0.2557063757013145
  (0, 2690)	0.219914470878468
  (0, 3303)	0.34816470617693457
  (0, 3574)	0.27930275383930203
  (0, 3932)	0.3668054859566594
  (0, 4294)	0.28401788033142783
  (0, 6409)	0.30807472481424736
  (0, 6720)	0.1985135703927375
  (0, 10492)	0.10794659665130096
  (0, 10526)	0.14330676349927227
  (0, 12319)	0.15029686659477812
  (0, 13155)	0.1851971378263286
  (0, 13379)	0.17604495063358377
  (0, 14476)	0.040989465772714374
  (0, 14757)	0.14991206717365188
  (0, 16506)	0.17572000214949124
  (0, 17610)	0.16184123155447802
  (0, 20185)	0.21125401562597
  (0, 22410)	0.23706559592158974
  (0, 24842)	0.18807609404761044
  (1, 3214)	0.38762500063537253
  (1, 3476)	0.2684883165137783
  (1, 3520)	0.3055102333035903
  (1, 4579)	0.3551892063143451
  (1, 5568)	0.35597012931701444
  :	:
  (25600, 415)	0.20281632583776385
  (25600, 947)	0.293514282175713
  (25600, 2719)	0.20857924740973227
  (25600, 2789)	0.09742419939777437
  (25600, 2975)	0.17870445384063285
  (25600, 3366)	0

In [26]:
model=LogisticRegression(max_iter=1000)


In [27]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Model Evalluation

In [33]:
#accutacy on training data
x_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(x_train_prediction,y_train)

In [31]:
print('Accuracy score of the training data : ', training_data_accuracy)#how much our model is correct

Accuracy score of the training data :  0.8868403577985235


In [34]:
#accuracy on test data
x_test_prediction=model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_prediction,y_test)

In [35]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7995625683486955


In [37]:
import pickle
#saving the training model

In [38]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))#traines model is saved as trained_model.sav

In [40]:
loaded_model=pickle.load(open('trained_model.sav','rb'))#rb-reading the file

In [53]:
x_new=x_test[0]
print(x_new)
prediction=loaded_model.predict(x_new)
print(prediction)

  (0, 3297)	0.17888427945924204
  (0, 4992)	0.16933692962285257
  (0, 5027)	0.10968655400593949
  (0, 5074)	0.1903822503687202
  (0, 5202)	0.3830610957155983
  (0, 7189)	0.1119085971800969
  (0, 7509)	0.3855189533008186
  (0, 7568)	0.13319030584929206
  (0, 12578)	0.45709941280414895
  (0, 12960)	0.0946628664081878
  (0, 13682)	0.12795735922371654
  (0, 14012)	0.13448668056112656
  (0, 14476)	0.030162138805774372
  (0, 15341)	0.1135256112826014
  (0, 17241)	0.3025164059282707
  (0, 17486)	0.11203343369900436
  (0, 17945)	0.1731957837089093
  (0, 21246)	0.2326890046754074
  (0, 21608)	0.1671064014866801
  (0, 23642)	0.26991417852313315
  (0, 25014)	0.10138877708648651
[-1.]


In [54]:
if(prediction[0]==0):
  print('The tweet is neutral')
elif(prediction[0]==1):
  print('The tweet is positive')
else:
  print('The tweet is negative')


The tweet is negative
